0 - create dataset  
1 - run experiment  
2 - plot counterfactuals  
3 - plot embeddings  
4 - plot results  

In [1]:
import shutil
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
# Set patient and record
osservazione_1 = "chb04_28"
osservazione_2 = "chb03_02"
osservazione_3 = "chb04_08"

### CONTROLLARE SE HANNO TUTTI GLI STESSI ELETTRODIIIIIIIIIIIIIIIII!!!

shutil.copyfile(f"EEG_data/dataset_{osservazione_1}.json", "data/datasets/gcs/grafo_corr_sliding_1.json")
shutil.copyfile(f"EEG_data/dataset_{osservazione_2}.json", "data/datasets/gcs/grafo_corr_sliding_2.json")
shutil.copyfile(f"EEG_data/dataset_{osservazione_3}.json", "data/datasets/gcs/grafo_corr_sliding_3.json")

'data/datasets/gcs/grafo_corr_sliding_3.json'

In [3]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache dataset

In [4]:
remove_cache_dataset = True

if remove_cache_dataset:
    folder = ".\data\cache\datasets"

    for file in os.listdir(folder):
        if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
            cache_path = os.path.join(folder, file)
            if os.path.isfile(cache_path):
                os.remove(cache_path)
                print(f"Removed: {cache_path}")

Removed: .\data\cache\datasets\GCS-295d7312ce6db4f307eb0c0f459dfbe1


## Run Gretel experiment

In [5]:
# NOTA: Se si blocca, andare a vedere se ci sono dei file OracleTorch provenienti dal futuro

In [6]:
path = 'config/GCS-GCN.jsonc'
# path = 'config/GCS-SVM.jsonc'
# path = 'config/GCS-KNN.jsonc'

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# Additional stuff
# context.logger.info(f"graph_ids: {eval_manager._evaluators[0].get_instance_and_counterfactual_ids()}") # NON È STATO SISTEMATO PER CONTEMPLARE IL CASO DI PIU' PAZIENTI
context.logger.info(f"graph_metrics: {eval_manager._evaluators[0].get_instance_and_counterfactual_graph_metrics()}")

Generating context for: config/GCS-GCN.jsonc
2025-03-06 11:37:41,668 | INFO | 22496 - Executing: config/GCS-GCN.jsonc Run: -1
2025-03-06 11:37:41,720 | INFO | 22496 - Creating the evaluation manager.......................................................
2025-03-06 11:37:41,783 | INFO | 22496 - Creating the PAIRED evaluators...............................................................
2025-03-06 11:37:41,973 | INFO | 22496 - Creating: GCS-295d7312ce6db4f307eb0c0f459dfbe1
2025-03-06 11:37:41,973 | INFO | 22496 - Instantiating: src.dataset.generators.gcs.GCS
2025-03-06 11:37:52,335 | INFO | 22496 - Saved: GCS-295d7312ce6db4f307eb0c0f459dfbe1
2025-03-06 11:37:52,415 | INFO | 22496 - Created: GCS-295d7312ce6db4f307eb0c0f459dfbe1
2025-03-06 11:37:52,462 | INFO | 22496 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-03-06 11:37:52,479 | INFO | 22496 - Instantiating: torch.optim.RMSprop
2025-03-06 11:37:52,479 | INFO | 22496 - Instantiating: torch.nn.CrossEntropyLoss
2025-03-06 11:37:5

In [7]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [8]:
import datetime

now = datetime.datetime.now()
print("Ultima esecuzione completa:", now.strftime("%d/%m/%Y, ore %H:%M"))

Ultima esecuzione completa: 06/03/2025, ore 11:57


# DEVO RICORDARMI DI FAR SI' CHE IL CONTROFATTUALE VENGA CERCATO SOLO NEI RECORD DEL PAZIENTE CORRETTO